In [10]:
from nltk.tokenize import word_tokenize
import nltk

In [11]:
import operator 
import json
from collections import Counter
import schedule

In [12]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
# ML Libraries
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fernandodelgadoteran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
#one global parameter
stop_words = set(stopwords.words('english'))

In [15]:
def load_tweets(filename, cols):
    dataset = pd.read_csv(filename, encoding='latin-1')
    dataset.columns = cols
    return dataset

In [16]:
twitter = load_tweets('input/training.1600000.processed.noemoticon.csv', ['target', 't_id', 'created_at', 'query', 'user', 'text'])

In [17]:
#twitter2 = twitter.sample(n=100_000, random_state=1).reset_index()

In [18]:
twitter.head()

,target,t_id,created_at,query,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [19]:
#twitter = twitter.drop(columns=['index'])
#twitter2.head()

In [20]:
twitter.shape

(1599999, 6)

In [21]:
#For processing @-mentions, emoticons, URLs and #hash-tags as individual tokens.

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [22]:
#vectorize data
def get_feature_vector(train_fit):
    vector = TfidfVectorizer(sublinear_tf=True)
    vector.fit(train_fit)
    return vector

In [23]:
def to_string(sentiment):
    if sentiment == 0:
        return "Negative"
    elif sentiment == 2:
        return "Neutral"
    else:
        return "Positive"

In [24]:
''' 
def preprocess_tweet_text(tweet):
    for i in tweet:
        i.lower()
        # Remove urls
        tweet = re.sub(r"http\S+|www\S+|https\S+", '', i, flags=re.MULTILINE)
        # Remove user @ references and '#' from tweet
        tweet = re.sub(r'\@\w+|\#','', i)
        # Remove punctuations
        tweet = tweet.translate(str.maketrans('', '', string.punctuation))
        # Remove stopwords
        tweet_tokens = word_tokenize(tweet)
        filtered_words = [w for w in tweet_tokens if not w in stop_words]

        #ps = PorterStemmer()
        #stemmed_words = [ps.stem(w) for w in filtered_words]
        #lemmatizer = WordNetLemmatizer()
        #lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]

        return " ".join(filtered_words)
        '''

In [25]:
def preprocess_tweet_text(tweet):
    tweet.lower()
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#','', tweet)
    # Remove punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    
    #ps = PorterStemmer()
    #stemmed_words = [ps.stem(w) for w in filtered_words]
    #lemmatizer = WordNetLemmatizer()
    #lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
    
    return " ".join(filtered_words)

In [26]:
#Preprocess data
twitter.text = twitter['text'].apply(preprocess_tweet_text)

In [27]:
def remove_unwanted_cols(dataset, cols):
    for col in cols:
        del dataset[col]
    return dataset

In [28]:
n_dataset = remove_unwanted_cols(twitter, ['t_id', 'created_at', 'query', 'user'])

In [29]:
# Same tf vector will be used for Testing sentiments on unseen trending data
tf_vector = get_feature_vector(np.array(twitter.iloc[:, 1]).ravel())
X = tf_vector.transform(np.array(twitter.iloc[:, 1]).ravel())
y = np.array(twitter.iloc[:, 0]).ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [30]:
# Training Naive Bayes model
NB_model = MultinomialNB()
NB_model.fit(X_train, y_train)
y_predict_nb = NB_model.predict(X_test)
print(accuracy_score(y_test, y_predict_nb))


0.768521875


In [31]:
# Training Logistics Regression model
LR_model = LogisticRegression(solver='lbfgs')
LR_model.fit(X_train, y_train)
y_predict_lr = LR_model.predict(X_test)
print(accuracy_score(y_test, y_predict_lr))

0.78778125


/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# API Analysis

In [ ]:
#Librerias matematicas
import pandas as pd
import numpy as np
#Libreria representacion
import matplotlib.pyplot as plt
import seaborn as sns
#Librerias de twitter
import tweepy
from tweepy import OAuthHandler
#Librerias tratar inet
import json
import re
import os
from dotenv import load_dotenv
from collections import Counter
import joblib

import time
import csv
#Representar mapas
import folium
#Representar imagenes
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
#Vectorizar,polarizar y tokenizar palabras
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

#Funciones delacasa
#from src.inputs import *

In [1]:
from __future__ import unicode_literals
CONSUMER_KEY = 
CONSUMER_SECRET = 
ACCESS_TOKEN = 
ACCESS_SECRET = 
 
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

user=api.me()
print(user.name,",", user.screen_name)

SyntaxError: invalid syntax (<ipython-input-1-d96ee40a68e7>, line 2)

In [ ]:
start_time = time.time()
sleep_on_rate_limit=False
max_id = -300000
tweetsPerQry = 1000  
search_words = '(Depressed OR Suicide OR Unhappy OR pain OR greatful OR thankful)'
date_since = "2020-09-29" 

# append data
csvFile = open(search_words + '_data4.csv', 'w', encoding='utf-8')

#csv Writer
csvWriter = csv.writer(csvFile)

#get data
tweet_num = 0
for tweet in tweepy.Cursor(
                        api.search,
                        q=search_words + ' -RT',
                        count=tweetsPerQry,
                        lang="en", 
                        since=date_since, max_id=str(max_id - 1)).items(9999999999):
    if tweet.place is not None:
        try:
            print ('tweet number: {}'.format(tweet_num), tweet.text, tweet.place.full_name)
            # write data
            csvWriter.writerow([tweet.created_at,
                                tweet.user.screen_name,
                                tweet.user.location,
                                tweet.user.followers_count,
                                tweet.user.friends_count,
                                tweet.text,
                                tweet.place.bounding_box.coordinates,
                                tweet.place.full_name,
                                tweet.place.country,
                                tweet.place.country_code,
                                tweet.place.place_type])
            tweet_num += 1
            
        except :
            continue

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#Defino las columnas que voy a utilizar y paso la informacion a un CSV
col_names=['date','user_name','user_loc','followers','friends','message','bbox_coords',\
           'full_name','country','country_code','place_type']
df2= pd.read_csv('(Depressed OR Suicide OR Unhappy OR pain OR happy OR greatful OR thankful)_data4.csv
, names=col_names)
print(df2.shape)
df2.head()
#Tras un exitoso web scrapping consigo rascar 2300 tweets